In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "Guia Spark")

**1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,
fecha). Se pide resolver utilizando PySpark:**
- A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
- B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
- C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido)

In [23]:
notas = sc.textFile("ej1.csv").map(lambda x: x.split(','))
notas.take(10)

[['10042', 'Quimica', '5', '02/17/2018'],
 ['10074', 'Algo1', '1', '12/15/2017'],
 ['10099', 'Analisis II', '2', '12/02/2018'],
 ['10024', 'Algo2', '7', '08/15/2017'],
 ['10098', 'Taller de programacion', '6', '05/20/2017'],
 ['10015', 'Analisis II', '5', '09/09/2018'],
 ['10037', 'Quimica', '1', '04/21/2019'],
 ['10013', 'Quimica', '10', '05/29/2019'],
 ['10062', 'Analisis II', '9', '04/12/2017'],
 ['10060', 'Algo3', '4', '10/11/2017']]

In [24]:
#A
#  0       1        2      3
#[ padron, materia, nota, fecha ]
aprobadas_ultimos_2_anios = notas.filter(lambda x: (int(x[2])>=4) & (x[3]>"01/01/2017"))
aprobadas_ultimos_2_anios.map(lambda x: x[0]).distinct().count()


101

In [25]:
#B
#armo un RDD de la forma [padron, [nota, 1]]
alumno_nota = notas.map(lambda x: [x[0], [int(x[2]), 1]])
alumno_nota.reduceByKey(lambda x,y: [x[1][0] + y[1][0], x[1][1] + y[1][1]])
alumno_promedio = alumno_nota.map(lambda x: [x[0], x[1][0]/x[1][1]])
alumno_promedio.take(5)

[['10042', 5.0],
 ['10074', 1.0],
 ['10099', 2.0],
 ['10024', 7.0],
 ['10098', 6.0]]

In [26]:

nombres = sc.textFile("ej1_nombres.csv").map(lambda x: x.split(','))
nombres.take(5)

[['10070', 'Esmaria Koppelmann'],
 ['10000', 'Lu Peinke'],
 ['10029', 'Kristyn Timlin'],
 ['10000', 'Phillip Boxer'],
 ['10051', 'Adrea Le Borgne']]

In [31]:
#C
alumno_mejor_promedio = alumno_promedio.reduce(lambda x,y: x if x[1]>y[1] else y)
nombre_alumno_mejor_promedio = nombres.filter(lambda x: x[0] == alumno_mejor_promedio[0])
nombre_alumno_mejor_promedio.collect()

[['10002', 'Kai Jammet'],
 ['10002', 'Arney Lathwell'],
 ['10002', 'Karel Richel'],
 ['10002', 'Eudora De Micoli'],
 ['10002', 'Keefe MacPaik'],
 ['10002', 'Marleen Loxdale'],
 ['10002', 'Prent Silliman'],
 ['10002', 'Kelby Leppard'],
 ['10002', 'Neal Fenne'],
 ['10002', 'Mario Frowen'],
 ['10002', 'Dedra Coston'],
 ['10002', 'Carol Whitman'],
 ['10002', 'Salomo Valeri'],
 ['10002', 'Cchaddie Whitehurst'],
 ['10002', 'Amory Gleadhall'],
 ['10002', 'Lesley Crowhurst'],
 ['10002', 'Dominic Giraudoux'],
 ['10002', 'Amalie Gile'],
 ['10002', 'Rosalinda MacAdie']]

**2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:**
- A. Cuál es el producto más vendido.
- B. Cuál es la categoría de productos más vendida.
- C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
- D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
- E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [35]:
ventas = sc.textFile("ej2.csv").map(lambda x: x.split(','))
productos = sc.textFile("ej2_productos.csv").map(lambda x: x.split(','))
ventas.take(3)

[['11/04/2019', '20', '338'],
 ['12/09/2019', '1', '412'],
 ['07/10/2019', '61', '395']]

In [34]:
productos.take(3)

[['1', 'quis tortor id nulla', 'Computacion'],
 ['2', 'et tempus semper est quam', 'Comida'],
 ['3', 'sed tristique in tempus sit amet sem', 'Computacion']]

In [41]:
#A
cant_vendidos = ventas.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y)
cant_vendidos.reduce(lambda x,y: x if x[1]>=y[1] else y)

('71', 17)

In [54]:
#B
ventas_ind_prod = ventas.map(lambda x: (x[1],1))
ventas_cat = productos.map(lambda x: (x[0],x[2])).join(ventas_ind_prod)
ventas_x_cat = ventas_cat.map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x,y: x+y)
ventas_x_cat.reduce(lambda x,y: x if x[1]>= y[1] else y)[0]

'Utileria'

In [65]:
#C
cant_vendidos_c_descr = productos.map(lambda x: (x[0], x[1])).join(cant_vendidos)
cant_vendidos_c_descr.map(lambda x: [x[0],x[1][0], x[1][1]]).takeOrdered(5, lambda x: -x[2])

[['71', 'nam congue risus semper porta volutpat quam pede', 17],
 ['65', 'id pretium iaculis diam', 16],
 ['17', 'mauris eget massa tempor', 15],
 ['68', 'diam vitae quam suspendisse potenti nullam porttitor lacus at', 15],
 ['77',
  'est lacinia nisi venenatis tristique fusce congue diam id ornare',
  15]]

Como takeOrdered devuelve una lista! Tenemos que hacer el Map para ordenarlo de la manera que lo queremos antes.